# Initial setup for modelling

In [ ]:
#!pip list

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pandas as pd
from math import sqrt
import statsmodels.api as sm
from datetime import datetime, timedelta
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
import itertools
import numpy as np
import pickle
import warnings
warnings.filterwarnings('ignore')

In [ ]:
file = open('store_v2.p', 'rb')
store = pickle.load(file)
file.close()

In [ ]:
store_ids = store.keys()

## Defining model kpis

In [ ]:
# Calculate KPIs

def calc_kpis(actuals, predictions, seasonality=7):
    
    #Experemential
    try:
        actuals.reset_index(drop=True,inplace=True)
        predictions.reset_index(drop=True,inplace=True)
    except:
        pass
    
    mae_kpi = mean_absolute_error(actuals, predictions)
    
    rmse_kpi = sqrt(mean_squared_error(actuals, predictions))
    
    mase_kpi = (mean_absolute_error(actuals, predictions) / 
                mean_absolute_error(actuals[seasonality:], actuals[:-seasonality]))
    
    #smape_kpi = np.sum(np.absolute(actuals - predictions)/((np.absolute(actuals)+np.absolute(predictions))/2))*(100/len(predictions))
    
    smape_kpi=np.sum(np.absolute(actuals - predictions)/((np.absolute(actuals)+np.absolute(predictions))/2))*(100/len(predictions))
    
    return mae_kpi, rmse_kpi, mase_kpi, smape_kpi

In [ ]:
def store_kpis(store_id,model_name,kpis):
    
    model_performance.loc[(store_id,model_name),["mae"]] = kpis[0] # "mae_kpi"
    model_performance.loc[(store_id,model_name),["rmse"]] = kpis[1] # rmse_kpi
    model_performance.loc[(store_id,model_name),["mase"]] = kpis[2] # mase_kpi
    model_performance.loc[(store_id,model_name),["smape"]] = kpis[3] # smape_kpi    

## Setup df to store model performances

In [ ]:
models = ["bl_naive2", "sarima", "prophet"]
multi_index = [store_ids,models]
kpis = ["mae","rmse","mase","smape","rel_mase", "rel_smape", "owa","rank"]
model_performance = pd.DataFrame(index=pd.MultiIndex.from_product(multi_index, names=['store', 'model']), columns=kpis)

In [ ]:
model_performance

# Models

## Baseline Models

### SNAIVE2

In [ ]:
# SNAIVE2 Baseline Model (S)ARIMA(X)(0,0,0)x(0,1,0)
def bl_naive2(data):
    
    data_train = data.loc[data["train_set"]==True,["Sales"]]
    data_test = data.loc[data["train_set"]==False,"Sales"]
    
    pred_start = data_train.index.max() + pd.DateOffset(1)
    pred_end = data_test.index.max()
    
    model = sm.tsa.statespace.SARIMAX(
        data_train,
        order=(0,0,0),
        seasonal_order=(0,1,0,7)
    ).fit()
    
    y_hat = model.get_prediction(
        start=pred_start,
        end=pred_end,
        dynamic=False
    )

    return dict(
        pred = y_hat.predicted_mean,
        kpis = calc_kpis(data_test, y_hat.predicted_mean, 7)
    )

In [ ]:
def apply_bl_naive2(data):
    store_result = bl_snaive2(store[store_id])
    store_kpis(store_id,"bl_naive2",store_result["kpis"])    

In [ ]:
#TESTING
#bl_snaive2(store[13])

In [ ]:
%%time
for store_id in store_ids:
    print("Applying NAIVE2 for store #"+str(store_id)+" out of "+ str(len(store_ids)))
    apply_bl_naive2(store[store_id])

In [ ]:
#model_performance

### SARIMA

In [ ]:
# Step 1: Define possible combinations
# Use only 12 combinations
## ARIMA(2, d, 2)(1, D, 1)
## ARIMA(0, d, 0)(0, D, 0)
## ARIMA(1, d, 0)(1, D, 0)
## ARIMA(0, d, 1)(0, D, 1)

#[(2, d, 2), (1, D, 1, 7)],
#[(0, d, 0), (0, D, 0, 7)],
#[(1, d, 0), (1, D, 0, 7)],
#[(0, d, 1), (0, D, 1, 7)],
def create_model_cfgs():
    return(np.array([
    [(2, 0, 2), (1, 0, 1, 7)],
    [(2, 1, 2), (1, 1, 1, 7)],
    [(2, 0, 2), (1, 1, 1, 7)],
    [(2, 1, 2), (1, 0, 1, 7)],

    [(0, 0, 0), (0, 0, 0, 7)],
    [(0, 1, 0), (0, 1, 0, 7)],
    [(0, 0, 0), (0, 1, 0, 7)],
    [(0, 1, 0), (0, 0, 0, 7)],

    [(1, 0, 0), (1, 0, 0, 7)],
    [(0, 1, 1), (0, 1, 1, 7)],
    [(0, 0, 1), (0, 1, 1, 7)],
    [(0, 1, 1), (0, 0, 1, 7)]]))

In [ ]:
# Step 2: Define SARIMA function to get AIC
def simulate_sarima(data, cfg):
    print("simulate "+str(cfg))
    order, sorder = cfg
    data_train = data.loc[data["train_set"]==True,["Sales"]]
    try:
        fit = sm.tsa.statespace.SARIMAX(data_train, order=order, seasonal_order=sorder).fit()
        return (cfg, fit.aic)
    except:
        return None

In [ ]:
# Step 3: Run all possible combinations and return best configuration
def find_best_parameters(data):
    model_cfgs = create_model_cfgs()
    #scores = list()
    #for cfg in model_cfgs:
    #    scores.append(simulate_sarima(data, cfg))
        
    executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
    tasks = (delayed(simulate_sarima)(data, cfg) for cfg in model_cfgs)
    scores = executor(tasks)
    
    # Step 4: Select best model cfg
    scores = [r for r in scores if r != None] #clean Nones
    scores.sort(key=lambda x: x[1])
    params, aic = scores[0]
    return params

In [ ]:
# Step 5: Apply SARIMA with best parameters
def sarima(data,y="Sales"):
    
    print("Finding best parameters to forecast " + y)
    b_pdq, b_spdq = find_best_parameters(data) # find best parameters
    
    print("Best parameters have been found:")
    print(str(b_pdq) + str(b_spdq))
    
    data_train = data.loc[data["train_set"]==True,[y]]
    data_test = data.loc[data["train_set"]==False,y]
    
    pred_start = data_train.index.max() + pd.DateOffset(1)
    pred_end = data_test.index.max()
    
    print("Fitting model")
    model = sm.tsa.statespace.SARIMAX(
        data_train,
        order=b_pdq,
        seasonal_order=b_spdq
    ).fit()
    
    print("Forecasting future")
    y_hat = model.get_prediction(
        start=pred_start,
        end=pred_end,
        dynamic=False
    )
    print("DONE")
    return dict(
        pred = y_hat.predicted_mean,
        kpis = calc_kpis(data_test, y_hat.predicted_mean, 7)
    )    

In [ ]:
def apply_sarima(data,y="Sales"):
    store_result = sarima(store[store_id],y)
    store_kpis(store_id,"sarima",store_result["kpis"])   

In [ ]:
#TESTING
#%%time
#results = apply_sarima(store[13],"Sales")

In [ ]:
%%time
#for store_id in store_ids:
for store_id in store_ids:
    print("Applying SARIMA for store #"+str(store_id)+" out of "+ str(len(store_ids)))
    apply_sarima(store[store_id])

In [ ]:

#model_performance

# Models with regressors

## FB Prophet

In [ ]:
from fbprophet import Prophet

In [ ]:
# Credits to https://www.kaggle.com/elenapetrova/time-series-analysis-and-forecasts-with-prophet
def get_holiday_df(df):
    state_dates = df[(df.StateHoliday == 'a') | (df.StateHoliday == 'b') & (df.StateHoliday == 'c')].loc[:, 'Date'].values
    school_dates = df[df.SchoolHoliday == 1].loc[:, 'Date'].values
    state = pd.DataFrame({'holiday': 'state_holiday',
                      'ds': pd.to_datetime(state_dates)})
    school = pd.DataFrame({'holiday': 'school_holiday',
                          'ds': pd.to_datetime(school_dates)})
    return pd.concat((state, school))

In [ ]:
def fb_univariate(df, y, fb_holidays):
    
    fb_train = df.loc[df.train_set==True,["ds",y]]
    fb_train.columns = ("ds","y")
    
    m = Prophet(holidays = fb_holidays, weekly_seasonality=True)
    m.fit(fb_train)
    
    future = m.make_future_dataframe(periods=91,include_history = False)
    forecast = m.predict(future)
    return forecast["yhat"]

In [ ]:
def fb_prophet(df, exog=None):
    
    # Splitting up train and test set
    fb_train = df.loc[df.train_set==True,]
    fb_test = df.loc[df.train_set==False,]
    
    # Get holidays
    fb_holidays = get_holiday_df(fb_train)
    
    # Rename for forecasting 
    fb_train.rename(columns={"Date": "ds", "Sales": "y"}, inplace=True)
    
    # Modeling
    m = Prophet(holidays = fb_holidays, weekly_seasonality=True)
    
    # Adding regressors
    if exog is not None:
        for ex in exog:
            m.add_regressor(ex)

        
    m.fit(fb_train)
    
    # Create future dataframe
    future = m.make_future_dataframe(periods=91,include_history = False) #+1 to get the 90 days
    
    # Predicting regressors first
    print("Predicting future values for exog regressors")
    if exog is not None:
        for ex in exog:
            future[ex] = fb_univariate(fb_train,ex,fb_holidays)
    print("--done")
    
    # Run forecast
    print("Start with forecasting")
    forecast = m.predict(future)
    print("--done")
    return dict(
        pred = forecast["yhat"],
        kpis = calc_kpis(fb_test["Sales"] , forecast["yhat"],7)
    )

In [ ]:
def apply_fb_prophet(df,exog=None):
    store_result = fb_prophet(store[store_id],exog=None)
    store_kpis(store_id,"prophet",store_result["kpis"])    

In [ ]:
%%time
#for store_id in [13]:
for store_id in store_ids:
    print("Applying Prophet for store #"+str(store_id)+" out of "+ str(len(store_ids)))
    apply_fb_prophet(store[store_id],["Customers","Promo"])

In [ ]:
#model_performance

# Calculate OWA performance and ranks

In [ ]:
import pickle
pickle.dump(model_performance, open("model_performance.p", "wb"))

In [ ]:
for store_id in [1,2,3]:
    for model in ["bl_snaive2", "bl_sarima", "prophet", "sarimax"]:
        
        model_performance.loc[(store_id, model),["rel_mase"]] = (
            model_performance.loc[(store_id, model),["mase"]].values/
            model_performance.loc[(store_id,"bl_snaive2"),["mase"]].values
        )
    
        model_performance.loc[(store_id, model),["rel_smape"]] = (
            model_performance.loc[(store_id, model),["smape"]].values/
            model_performance.loc[(store_id,"bl_snaive2"),["smape"]].values
        )
        
        model_performance.loc[(store_id, model),["owa"]] = (
            model_performance.loc[(store_id, model),["rel_mase"]].values+
            model_performance.loc[(store_id,"bl_snaive2"),["rel_smape"]].values
        )/2 

In [ ]:
for store_id in [1,2,3]:
    model_performance.loc[(store_id,),["rank"]] = model_performance.loc[(1,),["owa"]].rank(method='min').values
    

In [ ]:
model_performance

In [ ]:
pickle.dump(model_performance, open("model_performance_done.p", "wb"))